In [1]:
import json
with open("/Users/vishwajithp/python chatbot/intents.json",'r') as f:
    intents=json.load(f)
print(intents)

 

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'responses': ['Hey :-)', 'Hello, thanks for visiting', 'Hi there, what can I do for you?', 'Hi there, how can I help?']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later, thanks for visiting', 'Have a nice day', 'Bye! Come back again soon.']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", "Thank's a lot!"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'items', 'patterns': ['Which items do you have?', 'What kinds of items are there?', 'What do you sell?'], 'responses': ['We sell coffee and tea', 'We have coffee and tea']}, {'tag': 'payments', 'patterns': ['Do you take credit cards?', 'Do you accept Mastercard?', 'Can I pay with Paypal?', 'Are you cash only?'], 'responses': ['We accept VISA, Mastercard and Paypal', 'We accept most major credit cards, and Paypal']}, {'tag': 'd

In [52]:
import json
import nltk
import numpy as np
import random

#pytorch importing
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader

#The model ai
class NeuralNet(nn.Module):
     def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

     def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out




#creating tokenizer,stemmer and bagofowrds
from nltk.stem.porter import PorterStemmer
stemmer=PorterStemmer()
def tokenizer(sentence):
    return nltk.word_tokenize(sentence)
def stem(word):
    return stemmer.stem(word.lower())
def bag_of_words(tokenized_sentence,all_words):
    tokenized_sentence=[stem(w) for w in tokenized_sentence]

    bag=np.zeros(len(all_words),dtype=np.float32)
    for idx,w in enumerate(all_words):
        if w in tokenized_sentence:
            bag[idx]=1.0
    return bag


#creating instance of json file
with open("/Users/vishwajithp/python chatbot/intents.json",'r') as f:
    intents=json.load(f)

#creating empty arrays
all_words=[]
tags=[]
xy=[]
#creating tokenized words
for intent in intents['intents']:
    tag=intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w=tokenizer(pattern)
        all_words.extend(w)
        xy.append((w,tag))

#creating stemmed words removing punctuations and useless
ignore_words=['?','!','.',',']       
all_words=[stem(w) for w in all_words if w not in ignore_words]
all_words=sorted(set(all_words)) #Set makes sure repetition is removed
tags=sorted(set(tags))#getting tags


#creating training data
x_train=[]
y_train=[]
for (pattern_sentence,tag) in xy:
    bag=bag_of_words(pattern_sentence,all_words)
    x_train.append(bag)

#for y_Data
    label=tags.index(tag)
    y_train.append(label) 

x_train=np.array(x_train)
y_train=np.array(y_train)

class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples=len(x_train)
        self.x_data=x_train
        self.y_data=y_train
        

    #dataset[idx]
    def __getitem__(self,index):
        return self.x_data[index],self.y_data[index]


    def __len__ (self):
        return self.n_samples

#hyperparameters
batch_size = 8
hidden_size=8
output_size=len(tags)
input_size=len(x_train[0])
learning_rate=0.001
num_epochs=1000

dataset=ChatDataset()
train_loader=DataLoader(dataset=dataset,batch_size=batch_size,shuffle=True,num_workers=0)

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=NeuralNet(input_size,hidden_size,output_size).to(device)

#loss and optimiser
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)


for epoch in range(num_epochs):
    for (words,labels) in train_loader:
        words=words.to(device)
        labels=labels.to(device)


        #forward
        outputs=model(words)
        loss=criterion(outputs,labels)

        #backward and optimizer rates
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    if (epoch+1)%100==0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')


#saving data
data={
    "model_state":model.state_dict(),
    "input_size":input_size,
    "output_size":output_size,
    "hidden_size":hidden_size,
    "all_words":all_words,
    "tags":tags
}

FILE="data.pth"
torch.save(data,FILE)
print(f'training complete. file saved to {FILE}')


#using data and creating chat

data = torch.load(FILE)
input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()


bot_name="Hello"
print("Hello!! What do you wanna know type quit to exit")
while True:
    sentence=input("You : ")
    if sentence=="quit":
        break
    sentence=tokenizer(sentence)
    x=bag_of_words(sentence,all_words)
    x=x.reshape(1,x.shape[0])
    x=torch.from_numpy(x)


    output=model(x)
    _, predicted=torch.max(output,dim=1)
    tag=tags[predicted.item()]

    probs=torch.softmax(output,dim=1)
    prob=probs[0][predicted.item()]
    if prob.item()>0.7:
         for intent in intents["intents"]:
             if tag==intent["tag"]:
                 print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")
        
    

    
    

        




        




        
        


    
     

        
        
        





Epoch [100/1000], Loss: 0.6102
Epoch [200/1000], Loss: 0.0230
Epoch [300/1000], Loss: 0.0086
Epoch [400/1000], Loss: 0.0051
Epoch [500/1000], Loss: 0.0024
Epoch [600/1000], Loss: 0.0014
Epoch [700/1000], Loss: 0.0019
Epoch [800/1000], Loss: 0.0010
Epoch [900/1000], Loss: 0.0010
Epoch [1000/1000], Loss: 0.0001
final loss: 0.0001
training complete. file saved to data.pth
Hello!! What do you wanna know type quit to exit


You :  hi


Hello: Hi there, how can I help?


You :  timetable


Hello: The Timetable for today is:
9:00 AM - 10:00 AM: Science
10:00 AM - 10:15 AM: Break
10:15 AM - 11:15 AM: Maths 
11:15 AM - 11:30 AM: Break
11:30 AM - 12:30 PM: Social Studies (SST)
12:30 PM - 1:30 PM: Lunch Break
1:30 PM - 2:30 PM: English
2:30 PM - 2:45 PM: Break
2:45 PM - 3:45 PM: Revision
3:45 PM - 4:00 PM: Break
4:00 PM - 5:00 PM: Free Study or Extracurricular


You :  how are you


Hello: Hello, thanks for visiting


You :  bye


Hello: See you later, thanks for visiting


You :  quit
